# Work in C glutamicum

In [1]:
import numpy as np
from cobra.io import read_sbml_model

We load the model of C. glutamicum

In [2]:
model = read_sbml_model('seed_Seed196627_4.COBRA-sbml3.xml')
model

Name,seed_Seed196627_4
Memory address,0x07f608a434ac8
Number of metabolites,988
Number of reactions,1007
Number of groups,0
Objective expression,1.0*reacD3FA036D - 1.0*reacD3FA036D_reverse_06ea7
Compartments,"extracellular region, cytoplasm"


# Inserting enzyme 2.6.1.13

To make C. glutamicum produce putrescine via this new pathway we need to add the reactions of the enzymes 2.6.1.13 and 4.1.1.17

In [5]:
from cobra import Reaction, Metabolite

### 2.6.1.13

In [6]:
new_reaction_1 = Reaction('OAT')  # OAT -> (o)rnithine (a)mino(t)ransferase

We need to find the id's of the metabolites that are in this reaction: L-glutamate 5-semialdehyde, L-glutamate, Ornithine and 2-oxoglutarate

In [7]:
for metabolite in model.metabolites.query('L-glutamate 5-semialdehyde', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-glutamate 5-semialdehyde
MNXM245@MNXC3
N-acetyl-L-glutamate 5-semialdehyde
MNXM1062@MNXC3


In [8]:
model.metabolites.get_by_id('MNXM245@MNXC3') # L-glutamate 5-semialdehyde

Metabolite identifier,MNXM245@MNXC3
Name,L-glutamate 5-semialdehyde
Memory address,0x07f608a2ed8d0
Formula,C5H9NO3
Compartment,MNXC3
In 3 reaction(s),"mnxr99896c3, mnxr153703c3, mnxr103352c3"


In [12]:
for metabolite in model.metabolites.query('L-glutamate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-glutamate
MNXM741173@MNXC3
L-glutamate
MNXM741173@MNXC2
N-acetyl-L-glutamate
MNXM730@MNXC3
L-glutamate 5-semialdehyde
MNXM245@MNXC3
Tetrahydropteroyltri-L-glutamate
MNXM733072@MNXC3
5-Methyltetrahydropteroyltri-L-glutamate
MNXM733158@MNXC3
10-Formyl-THF-L-glutamate
MNXM33047@MNXC3
tetrahydropteroyltri-L-glutamate
MNXM733068@MNXC3
N-acetyl-L-glutamate 5-semialdehyde
MNXM1062@MNXC3


In [13]:
model.metabolites.get_by_id('MNXM741173@MNXC3') # L-glutamate

Metabolite identifier,MNXM741173@MNXC3
Name,L-glutamate
Memory address,0x07f1a0f851550
Formula,C5H8NO4
Compartment,MNXC3
In 43 reaction(s),"mnxr100300c2c3, mnxr137140c3, mnxr126234c3, mnxr100384c3, mnxr100277c3, mnxr103352c3, mnxr144502c3, mnxr95388c3, mnxr96228c3, mnxr105362c3, mnxr102221c3, mnxr100291c3, mnxr137143c3, mnxr153021c3, m..."


In [14]:
for metabolite in model.metabolites.query('ornithine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-ornithine
MNXM741175@MNXC3
N(2)-acetyl-L-ornithine
MNXM817@MNXC3


In [15]:
model.metabolites.get_by_id('MNXM741175@MNXC3') # L-ornithine

Metabolite identifier,MNXM741175@MNXC3
Name,L-ornithine
Memory address,0x07f1a0f73e6d8
Formula,C5H13N2O2
Compartment,MNXC3
In 3 reaction(s),"mnxr102221c3, mnxr102218c3, mnxr102137c3"


In [16]:
for metabolite in model.metabolites.query('oxoglutarate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

2-oxoglutarate
MNXM20@MNXC3


In [17]:
model.metabolites.get_by_id('MNXM20@MNXC3') # 2-oxoglutarate

Metabolite identifier,MNXM20@MNXC3
Name,2-oxoglutarate
Memory address,0x07f1a0f7e6128
Formula,C5H4O5
Compartment,MNXC3
In 16 reaction(s),"mnxr96230c3, mnxr126234c3, mnxr146406c3, mnxr95388c3, mnxr106443c3, mnxr96228c3, mnxr95852c3, mnxr100291c3, mnxr105310c3, mnxr96229c3, mnxr100086c3, mnxr95186c3, mnxr104301c3, mnxr105000c3, mnxr103..."


In [18]:
new_reaction_1.add_metabolites({model.metabolites.get_by_id('MNXM245@MNXC3'): -1, # L-glutamate 5-semialdehyde
                              model.metabolites.get_by_id('MNXM741173@MNXC3'): -1, # L-glutamate 
                              model.metabolites.get_by_id('MNXM741175@MNXC3'): 1, # L-ornithine
                              model.metabolites.get_by_id('MNXM20@MNXC3'): 1 # 2-oxoglutarate
                             })

In [19]:
print(new_reaction_1.build_reaction_string())

MNXM245@MNXC3 + MNXM741173@MNXC3 --> MNXM20@MNXC3 + MNXM741175@MNXC3


In [20]:
model.add_reactions([new_reaction_1])

In [21]:
model.reactions.OAT

Reaction identifier,OAT
Name,
Memory address,0x07f1a0f0a2dd8
Stoichiometry,MNXM245@MNXC3 + MNXM741173@MNXC3 --> MNXM20@MNXC3 + MNXM741175@MNXC3 L-glutamate 5-semialdehyde + L-glutamate --> 2-oxoglutarate + L-ornithine
GPR,
Lower bound,0.0
Upper bound,1000.0


In [22]:
with model:
    model.objective = model.reactions.OAT
    print(model.optimize().objective_value)  # We produce L-ornithine

1000.0


We add bounds

In [23]:
model.add_boundary(model.metabolites.get_by_id('MNXM741175@MNXC3'), type='demand') # Demand for L-ornithine

Reaction identifier,DM_MNXM741175@MNXC3
Name,L-ornithine demand
Memory address,0x07f1a0f0e49e8
Stoichiometry,MNXM741175@MNXC3 --> L-ornithine -->
GPR,
Lower bound,0
Upper bound,1000.0


## 4.1.1.17

In [24]:
new_reaction_2 = Reaction('ODC')  # ODC -> (o)rnithine (d)e(c)arboxylase

In [25]:
for metabolite in model.metabolites.query('putrescine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

putrescine
MNXM118@MNXC3


In [26]:
model.metabolites.get_by_id('MNXM118@MNXC3') # Putrescine

Metabolite identifier,MNXM118@MNXC3
Name,putrescine
Memory address,0x07f1a0f4e8668
Formula,C4H14N2
Compartment,MNXC3
In 1 reaction(s),mnxr95860c3


In [27]:
for metabolite in model.metabolites.query('CO2', 'name'):
    print(metabolite.name)
    print(metabolite.id)

CO2
MNXM13@MNXC3
CO2
MNXM13@MNXC2


In [28]:
model.metabolites.get_by_id('MNXM13@MNXC3') # CO2

Metabolite identifier,MNXM13@MNXC3
Name,CO2
Memory address,0x07f1a0f7981d0
Formula,CO2
Compartment,MNXC3
In 63 reaction(s),"mnxr134864c3, mnxr101443c3, mnxr146406c3, mnxr101662c3, mnxr96481c3, mnxr100389c3, mnxr105362c3, mnxr95851c3, mnxr134860c3, mnxr101665c3, mnxr145677c3, mnxr97180c3, mnxr134600c3, mnxr106658c3, mnxr..."


In [29]:
new_reaction_2.add_metabolites({model.metabolites.get_by_id('MNXM741175@MNXC3'): -1, # L-ornithine
                              model.metabolites.get_by_id('MNXM13@MNXC3'): 1, # CO2
                              model.metabolites.get_by_id('MNXM118@MNXC3'): 1 # # Putrescine
                             })

In [30]:
print(new_reaction_2.build_reaction_string())

MNXM741175@MNXC3 --> MNXM118@MNXC3 + MNXM13@MNXC3


In [31]:
model.add_reactions([new_reaction_2])

In [32]:
model.reactions.ODC

Reaction identifier,ODC
Name,
Memory address,0x07f1a0f0e4048
Stoichiometry,MNXM741175@MNXC3 --> MNXM118@MNXC3 + MNXM13@MNXC3 L-ornithine --> putrescine + CO2
GPR,
Lower bound,0.0
Upper bound,1000.0


In [33]:
with model:
    model.objective = model.reactions.OAT
    print("Maximum Putrescine yield: ", model.optimize().objective_value) # We produce putrescine

1000.0


In [34]:
model.add_boundary(model.metabolites.get_by_id('MNXM118@MNXC3'), type='demand') # Demand for putrescine

Reaction identifier,DM_MNXM118@MNXC3
Name,putrescine demand
Memory address,0x07f1a0f0e43c8
Stoichiometry,MNXM118@MNXC3 --> putrescine -->
GPR,
Lower bound,0
Upper bound,1000.0


The production of putrescine seems to just follow the upper bound. To investigate this further we looked into the fluxes in the original unmodified model

# Model fluxes check

In [35]:
modelcheck = read_sbml_model('seed_Seed196627_4.COBRA-sbml3.xml')


The model is checked for fluxes

In [ ]:
solution = modelcheck.optimize()
solution.fluxes

The fluxes look to be zero. We want to check if there are any fluxes that are not zero

In [ ]:
solution.fluxes[solution.fluxes != 0]

In [ ]:
All the fluxes are equal to zero. This makes the model very difficult to work with.

In [40]:
# We look up the model summary. This shows that there are no fluxes
model.summary()

As this model seems to be incomplete, we will try and insert the corresponding pathway in another production host, E. coli. 